In [1]:
if (typeof(Base.find_package("UnicodePlots")) == Nothing)
    println("Package Unicode not installed")
    using Pkg;
    Pkg.add("UnicodePlots")
end
using UnicodePlots

## Using Packages

In [2]:
using .Threads
using BenchmarkTools
using SparseArrays
using LinearAlgebra

## Number of Threads

In [3]:
nthreads()

4

## Including External Files|

In [4]:
include("global_curved_multithreading.jl")

plot_blocks (generic function with 1 method)

## Setting SBP operator Orders

In [5]:
SBPp = 6

6

## Reading .inp files and initializing boundary conditions

In [6]:
bc_map = [BC_DIRICHLET, BC_DIRICHLET, BC_NEUMANN, BC_NEUMANN, BC_JUMP_INTERFACE]
# 1 refers to Dirichlet boundary condition
# 2 refers to Neumann boundary condition
# 7 refers to Jump interface condition

5-element Array{Int64,1}:
 1
 1
 2
 2
 7

In [7]:
(verts, EToV, EToF, FToB, EToDomain) = read_inp_2d("../meshes/4_4_block.inp", bc_map=bc_map)

([0.5 -0.5 … 0.25 0.25; 0.5 0.5 … -0.5 -0.25], [4 7 … 9 6; 23 24 … 16 21; 24 20 … 20 16; 25 25 … 22 22], [1 5 … 9 29; 2 4 … 39 36; 3 6 … 30 40; 4 7 … 38 39], [1, 0, 1, 0, 0, 1, 0, 0, 0, 0  …  0, 1, 1, 0, 1, 0, 1, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [8]:
verts'

25×2 Array{Float64,2}:
  0.5    0.5 
 -0.5    0.5 
 -0.5   -0.5 
  0.5   -0.5 
  0.0    0.5 
 -0.5    0.0 
  0.0   -0.5 
  0.5    0.0 
  0.0    0.0 
  0.25   0.5 
  0.0    0.25
  0.25   0.0 
  0.5    0.25
  0.25   0.25
 -0.5    0.25
 -0.25   0.0 
 -0.25   0.5 
 -0.25   0.25
 -0.25  -0.5 
  0.0   -0.25
 -0.5   -0.25
 -0.25  -0.25
  0.5   -0.25
  0.25  -0.5 
  0.25  -0.25

In [9]:
EToV

4×16 Array{Int64,2}:
  4   7   9   8   1   5   9   8   2   6   9   5   3   7   9   6
 23  24  20  12  10  11  12  13  15  16  11  17  19  20  16  21
 24  20  12  23  13  10  11  12  17  15  16  11  21  19  20  16
 25  25  25  25  14  14  14  14  18  18  18  18  22  22  22  22

In [10]:
FToB

40-element Array{Int64,1}:
 1
 0
 1
 0
 0
 1
 0
 0
 0
 0
 1
 0
 1
 ⋮
 0
 0
 0
 1
 1
 0
 1
 0
 1
 0
 0
 1

In [11]:
EToDomain

16-element Array{Int64,1}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

In [12]:
(nelems, nfaces) = (size(EToV, 2), size(FToB, 1))
@show (nelems, nfaces)

(nelems, nfaces) = (16, 40)


(16, 40)

In [13]:
# This is needed to fix up points that should be on the boundary of the
# circle, but the mesh didn't quite put them there
for v in 1:size(verts, 2)
    x,y = verts[1,v], verts[2,v]
    if abs(hypot(x,y) - 1) < 1e-5
        Q = atan(y,x)
        verts[1,v], verts[2,v] = cos(Q), sin(Q)
    end
end


In [14]:
plot_connectivity(verts, EToV)

                                        connectivity
      ┌────────────────────────────────────────────────────────────────────────────────┐ 
    1 │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
      │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
      │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
      │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
      │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
      │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
      │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
      │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
      │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
      │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀

## Setting base mesh sizes in each direction

In [15]:
N1 = N0 = 16

16

In [16]:
EToN0 = zeros(Int64, 2, nelems)
EToN0[1,:] .= N0;
@show EToN0
EToN0[2,:] .= N1;
@show EToN0

EToN0 = [16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16; 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
EToN0 = [16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16; 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16]


2×16 Array{Int64,2}:
 16  16  16  16  16  16  16  16  16  16  16  16  16  16  16  16
 16  16  16  16  16  16  16  16  16  16  16  16  16  16  16  16

In [17]:
# Checking types and sizes 
@assert typeof(EToV) == Array{Int, 2} && size(EToV) == (4, nelems)
@assert typeof(EToF) == Array{Int, 2} && size(EToF) == (4, nelems)
@assert maximum(maximum(EToF)) == nfaces   # The maximum number of EToF element should equal the number of faces 

# Determine secondary arrays
# FToE : Unique Global Face to Element Number
#        (the i'th column of this stores the element numbers that share the
#        global face number i)
# FToLF: Unique Global Face to Element local face number
#        (the i'th column of this stores the element local face numbers that
#        shares the global face number i)
# EToO : Element to Unique Global Faces Orientation
#        (the i'th column of this stores the whether the element and global
#        face are oriented in the same way in physical memory or need to be
#        rotated)
# EToS : Element to Unique Global Face Side
#        (the i'th column of this stores whether an element is on the
#        plus side or minus side of the global face)


In [18]:
# connectivity arrays
(FToE, FToLF, EToO, EToS) = connectivityarrays(EToV, EToF)

([1 1 … 15 16; 0 4 … 16 0], [1 2 … 2 3; 0 4 … 4 0], Bool[1 1 … 1 1; 1 1 … 1 1; 1 1 … 1 1; 1 1 … 1 1], [1 1 … 2 2; 1 2 … 1 2; 1 1 … 2 1; 1 1 … 2 2])

## Forming exact solutions

In [19]:
Lx = maximum(verts[1,:])
@show Lx
Ly = maximum(abs.(verts[2,:]))
@show Ly

Lx = 0.5
Ly = 0.5


0.5

In [20]:
(kx, ky) = (2*π / Lx, 4*π / Ly)

(12.566370614359172, 25.132741228718345)

In [21]:
vex(x,y,e) = begin
    if EToDomain[e] == 1
        return cos.(kx * x) .* cosh.(ky * y)
    elseif EToDomain[e] == 2
        return 10 .+ cos.(kx * x) .* cosh.(ky * y)
    else
        error("invalid block")
    end
end

vex (generic function with 1 method)

In [22]:
vex_x(x,y,e) = begin
  if EToDomain[e] == 1
    return -kx * sin.(kx * x) .* cosh.(ky * y)
  elseif EToDomain[e] == 2
    return -kx * sin.(kx * x) .* cosh.(ky * y)
  else
    error("invalid block")
  end
end

vex_x (generic function with 1 method)

In [23]:
vex_y(x,y,e) = begin
  if EToDomain[e] == 1
    return ky * cos.(kx * x) .* sinh.(ky * y)
  elseif EToDomain[e] == 2
    return ky * cos.(kx * x) .* sinh.(ky * y)
  else
    error("invalid block")
  end
end

vex_y (generic function with 1 method)

In [24]:
vex_xx(x,y,e) = begin
  if EToDomain[e] == 1
    return -kx^2 * cos.(kx * x) .* cosh.(ky * y)
  elseif EToDomain[e] == 2
    return -kx^2 * cos.(kx * x) .* cosh.(ky * y)
  else
    error("invalid block")
  end
end

vex_xx (generic function with 1 method)

In [25]:
vex_xy(x,y,e) = begin
  if EToDomain[e] == 1
    return -kx * ky * sin.(kx * x) .* sinh.(ky * y)
  elseif EToDomain[e] == 2
    return -kx * ky * sin.(kx * x) .* sinh.(ky * y)
  else
    error("invalid block")
  end
end

vex_xy (generic function with 1 method)

In [26]:
vex_yy(x,y,e) = begin
  if EToDomain[e] == 1
    return ky^2 * cos.(kx * x) .* cosh.(ky * y)
  elseif EToDomain[e] == 2
    return ky^2 * cos.(kx * x) .* cosh.(ky * y)
  else
    error("invalid block")
  end
end

vex_yy (generic function with 1 method)

In [27]:
ϵ = zeros(4)

4-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0

In [28]:
lvl = 4

4

In [29]:
# generate base mesh size for each element
Nr = EToN0[1,:] * (2^(lvl - 1))
Ns = EToN0[2,:] * (2^(lvl - 1))

16-element Array{Int64,1}:
 128
 128
 128
 128
 128
 128
 128
 128
 128
 128
 128
 128
 128
 128
 128
 128

In [30]:
OPTYPE = typeof(locoperator(2,8,8))

NamedTuple{(:M̃, :F, :coord, :facecoord, :JH, :sJ, :nx, :ny, :Hf, :HfI, :τ, :bctype),Tuple{SparseMatrixCSC{Float64,Int64},NTuple{4,SparseMatrixCSC{Float64,Int64}},Tuple{Array{Float64,2},Array{Float64,2}},Tuple{Tuple{SubArray{Float64,1,Array{Float64,2},Tuple{Int64,Base.Slice{Base.OneTo{Int64}}},true},SubArray{Float64,1,Array{Float64,2},Tuple{Int64,Base.Slice{Base.OneTo{Int64}}},true},SubArray{Float64,1,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},Int64},true},SubArray{Float64,1,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},Int64},true}},Tuple{SubArray{Float64,1,Array{Float64,2},Tuple{Int64,Base.Slice{Base.OneTo{Int64}}},true},SubArray{Float64,1,Array{Float64,2},Tuple{Int64,Base.Slice{Base.OneTo{Int64}}},true},SubArray{Float64,1,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},Int64},true},SubArray{Float64,1,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},Int64},true}}},SparseMatrixCSC{Float64,Int64},NTuple{4,Array{Float64,1}},NTuple{4,Array{Float64,1}},NTuple{4,A

In [31]:
lop = Dict{Int64, OPTYPE}()

Dict{Int64,NamedTuple{(:M̃, :F, :coord, :facecoord, :JH, :sJ, :nx, :ny, :Hf, :HfI, :τ, :bctype),Tuple{SparseMatrixCSC{Float64,Int64},NTuple{4,SparseMatrixCSC{Float64,Int64}},Tuple{Array{Float64,2},Array{Float64,2}},Tuple{Tuple{SubArray{Float64,1,Array{Float64,2},Tuple{Int64,Base.Slice{Base.OneTo{Int64}}},true},SubArray{Float64,1,Array{Float64,2},Tuple{Int64,Base.Slice{Base.OneTo{Int64}}},true},SubArray{Float64,1,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},Int64},true},SubArray{Float64,1,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},Int64},true}},Tuple{SubArray{Float64,1,Array{Float64,2},Tuple{Int64,Base.Slice{Base.OneTo{Int64}}},true},SubArray{Float64,1,Array{Float64,2},Tuple{Int64,Base.Slice{Base.OneTo{Int64}}},true},SubArray{Float64,1,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},Int64},true},SubArray{Float64,1,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},Int64},true}}},SparseMatrixCSC{Float64,Int64},NTuple{4,Array{Float64,1}},NTuple{4,Array{Float64,1}}

In [32]:
@benchmark for e = 1:nelems
      # Get the element corners
      (x1, x2, x3, x4) = verts[1, EToV[:, e]]
      (y1, y2, y3, y4) = verts[2, EToV[:, e]]

      # Initialize the block transformations as transfinite between the corners
      ex = [(α) -> x1 * (1 .- α) / 2 + x3 * (1 .+ α) / 2,
            (α) -> x2 * (1 .- α) / 2 + x4 * (1 .+ α) / 2,
            (α) -> x1 * (1 .- α) / 2 + x2 * (1 .+ α) / 2,
            (α) -> x3 * (1 .- α) / 2 + x4 * (1 .+ α) / 2]
      exα = [(α) -> -x1 / 2 + x3 / 2,
             (α) -> -x2 / 2 + x4 / 2,
             (α) -> -x1 / 2 + x2 / 2,
             (α) -> -x3 / 2 + x4 / 2]
      ey = [(α) -> y1 * (1 .- α) / 2 + y3 * (1 .+ α) / 2,
            (α) -> y2 * (1 .- α) / 2 + y4 * (1 .+ α) / 2,
            (α) -> y1 * (1 .- α) / 2 + y2 * (1 .+ α) / 2,
            (α) -> y3 * (1 .- α) / 2 + y4 * (1 .+ α) / 2]
      eyα = [(α) -> -y1 / 2 + y3 / 2,
             (α) -> -y2 / 2 + y4 / 2,
             (α) -> -y1 / 2 + y2 / 2,
             (α) -> -y3 / 2 + y4 / 2]

      # For blocks on the circle, put in the curved edge transform
      if FToB[EToF[1, e]] == BC_JUMP_INTERFACE
        error("curved face 1 not implemented yet")
      end
      if FToB[EToF[2, e]] == BC_JUMP_INTERFACE
        error("curved face 2 not implemented yet")
      end
      if FToB[EToF[3, e]] == BC_JUMP_INTERFACE
        Q1 = atan(y1, x1)
        Q2 = atan(y2, x2)
        if !(-π/2 < Q1 - Q2 < π/2)
          Q2 -= sign(Q2) * 2 * π
        end
        ex[3] = (α) -> cos.(Q1 * (1 .- α) / 2 + Q2 * (1 .+ α) / 2)
        ey[3] = (α) -> sin.(Q1 * (1 .- α) / 2 + Q2 * (1 .+ α) / 2)
        β3 = (Q2 - Q1) / 2
        exα[3] = (α) -> -β3 .* sin.(Q1 * (1 .- α) / 2 + Q2 * (1 .+ α) / 2)
        eyα[3] = (α) -> +β3 .* cos.(Q1 * (1 .- α) / 2 + Q2 * (1 .+ α) / 2)
      end
      if FToB[EToF[4, e]] == BC_JUMP_INTERFACE
        Q3 = atan(y3, x3)
        Q4 = atan(y4, x4)
        if !(-π/2 < Q3 - Q4 < π/2)
          error("curved face 4 angle correction not implemented yet")
        end
        ex[4] = (α) -> cos.(Q3 * (1 .- α) / 2 + Q4 * (1 .+ α) / 2)
        ey[4] = (α) -> sin.(Q3 * (1 .- α) / 2 + Q4 * (1 .+ α) / 2)
        β4 = (Q4 - Q3) / 2
        exα[4] = (α) -> -β4 .* sin.(Q3 * (1 .- α) / 2 + Q4 * (1 .+ α) / 2)
        eyα[4] = (α) -> +β4 .* cos.(Q3 * (1 .- α) / 2 + Q4 * (1 .+ α) / 2)
      end

      # Create the volume transform as the transfinite blending of the edge
      # transformations
      xt(r,s) = transfinite_blend(ex[1], ex[2], ex[3], ex[4],
                                  exα[1], exα[2], exα[3], exα[4],
                                  r, s)
      yt(r,s) = transfinite_blend(ey[1], ey[2], ey[3], ey[4],
                                  eyα[1], eyα[2], eyα[3], eyα[4],
                                  r, s)


      metrics = create_metrics(SBPp, Nr[e], Ns[e], xt, yt)

      # Build local operators
      lop[e] = locoperator(SBPp, Nr[e], Ns[e], metrics, FToB[EToF[:, e]])
    end

BenchmarkTools.Trial: 
  memory estimate:  4.65 GiB
  allocs estimate:  1452353
  --------------
  minimum time:     1.539 s (17.91% GC)
  median time:      1.675 s (19.66% GC)
  mean time:        1.648 s (19.50% GC)
  maximum time:     1.704 s (20.61% GC)
  --------------
  samples:          4
  evals/sample:     1

In [33]:
@benchmark @threads for e = 1:nelems
      # Get the element corners
      (x1, x2, x3, x4) = verts[1, EToV[:, e]]
      (y1, y2, y3, y4) = verts[2, EToV[:, e]]

      # Initialize the block transformations as transfinite between the corners
      ex = [(α) -> x1 * (1 .- α) / 2 + x3 * (1 .+ α) / 2,
            (α) -> x2 * (1 .- α) / 2 + x4 * (1 .+ α) / 2,
            (α) -> x1 * (1 .- α) / 2 + x2 * (1 .+ α) / 2,
            (α) -> x3 * (1 .- α) / 2 + x4 * (1 .+ α) / 2]
      exα = [(α) -> -x1 / 2 + x3 / 2,
             (α) -> -x2 / 2 + x4 / 2,
             (α) -> -x1 / 2 + x2 / 2,
             (α) -> -x3 / 2 + x4 / 2]
      ey = [(α) -> y1 * (1 .- α) / 2 + y3 * (1 .+ α) / 2,
            (α) -> y2 * (1 .- α) / 2 + y4 * (1 .+ α) / 2,
            (α) -> y1 * (1 .- α) / 2 + y2 * (1 .+ α) / 2,
            (α) -> y3 * (1 .- α) / 2 + y4 * (1 .+ α) / 2]
      eyα = [(α) -> -y1 / 2 + y3 / 2,
             (α) -> -y2 / 2 + y4 / 2,
             (α) -> -y1 / 2 + y2 / 2,
             (α) -> -y3 / 2 + y4 / 2]

      # For blocks on the circle, put in the curved edge transform
      if FToB[EToF[1, e]] == BC_JUMP_INTERFACE
        error("curved face 1 not implemented yet")
      end
      if FToB[EToF[2, e]] == BC_JUMP_INTERFACE
        error("curved face 2 not implemented yet")
      end
      if FToB[EToF[3, e]] == BC_JUMP_INTERFACE
        Q1 = atan(y1, x1)
        Q2 = atan(y2, x2)
        if !(-π/2 < Q1 - Q2 < π/2)
          Q2 -= sign(Q2) * 2 * π
        end
        ex[3] = (α) -> cos.(Q1 * (1 .- α) / 2 + Q2 * (1 .+ α) / 2)
        ey[3] = (α) -> sin.(Q1 * (1 .- α) / 2 + Q2 * (1 .+ α) / 2)
        β3 = (Q2 - Q1) / 2
        exα[3] = (α) -> -β3 .* sin.(Q1 * (1 .- α) / 2 + Q2 * (1 .+ α) / 2)
        eyα[3] = (α) -> +β3 .* cos.(Q1 * (1 .- α) / 2 + Q2 * (1 .+ α) / 2)
      end
      if FToB[EToF[4, e]] == BC_JUMP_INTERFACE
        Q3 = atan(y3, x3)
        Q4 = atan(y4, x4)
        if !(-π/2 < Q3 - Q4 < π/2)
          error("curved face 4 angle correction not implemented yet")
        end
        ex[4] = (α) -> cos.(Q3 * (1 .- α) / 2 + Q4 * (1 .+ α) / 2)
        ey[4] = (α) -> sin.(Q3 * (1 .- α) / 2 + Q4 * (1 .+ α) / 2)
        β4 = (Q4 - Q3) / 2
        exα[4] = (α) -> -β4 .* sin.(Q3 * (1 .- α) / 2 + Q4 * (1 .+ α) / 2)
        eyα[4] = (α) -> +β4 .* cos.(Q3 * (1 .- α) / 2 + Q4 * (1 .+ α) / 2)
      end

      # Create the volume transform as the transfinite blending of the edge
      # transformations
      xt(r,s) = transfinite_blend(ex[1], ex[2], ex[3], ex[4],
                                  exα[1], exα[2], exα[3], exα[4],
                                  r, s)
      yt(r,s) = transfinite_blend(ey[1], ey[2], ey[3], ey[4],
                                  eyα[1], eyα[2], eyα[3], eyα[4],
                                  r, s)


      metrics = create_metrics(SBPp, Nr[e], Ns[e], xt, yt)

      # Build local operators
      lop[e] = locoperator(SBPp, Nr[e], Ns[e], metrics, FToB[EToF[:, e]])
    end

BenchmarkTools.Trial: 
  memory estimate:  4.65 GiB
  allocs estimate:  1452369
  --------------
  minimum time:     999.306 ms (40.13% GC)
  median time:      1.078 s (43.02% GC)
  mean time:        1.070 s (43.47% GC)
  maximum time:     1.126 s (46.39% GC)
  --------------
  samples:          5
  evals/sample:     1

Conclusing 1: Using @threads macro could improve performance for building local operators

In [34]:
time_LocalGlobalOperators_start = time()
(M, FbarT, D, vstarts, FToλstarts) = threaded_LocalGlobalOperators(lop, Nr, Ns, FToB, FToE, FToLF, EToO, EToS, (x) -> cholesky(Symmetric(x)))
time_LocalGlobalOperators_elapsed = time() - time_LocalGlobalOperators_start

2.2940001487731934

In [35]:
@show lvl

lvl = 4


4

In [36]:
M

SBPLocalOperator1{Float64,SuiteSparse.CHOLMOD.Factor{Float64}}([1, 16642, 33283, 49924, 66565, 83206, 99847, 116488, 133129, 149770, 166411, 183052, 199693, 216334, 232975, 249616, 266257], [3.807976433115568e-7, 1.6757718840580747e-6, 7.563502168786216e-7, 1.495123870251764e-6, 1.0988142121639435e-6, 1.2220175525452047e-6, 1.205250068947121e-6, 1.205250068947121e-6, 1.205250068947121e-6, 1.205250068947121e-6  …  1.205250068947121e-6, 1.205250068947121e-6, 1.205250068947121e-6, 1.205250068947121e-6, 1.2220175525452047e-6, 1.0988142121639435e-6, 1.495123870251764e-6, 7.563502168786216e-7, 1.6757718840580747e-6, 3.807976433115568e-7], [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5  …  -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25], [-0.5, -0.498046875, -0.49609375, -0.494140625, -0.4921875, -0.490234375, -0.48828125, -0.486328125, -0.484375, -0.482421875  …  -0.232421875, -0.234375, -0.236328125, -0.23828125, -0.240234375, -0.2421875, -0.244140625, -0.24609375, 

In [37]:
locfactors = M.F

16-element Array{SuiteSparse.CHOLMOD.Factor{Float64},1}:
 SuiteSparse.CHOLMOD.Factor{Float64}
type:    LLt
method:  supernodal
maxnnz:  0
nnz:     3602642
success: true

 SuiteSparse.CHOLMOD.Factor{Float64}
type:    LLt
method:  supernodal
maxnnz:  0
nnz:     3602642
success: true

 SuiteSparse.CHOLMOD.Factor{Float64}
type:    LLt
method:  supernodal
maxnnz:  0
nnz:     3602642
success: true

 SuiteSparse.CHOLMOD.Factor{Float64}
type:    LLt
method:  supernodal
maxnnz:  0
nnz:     3602642
success: true

 SuiteSparse.CHOLMOD.Factor{Float64}
type:    LLt
method:  supernodal
maxnnz:  0
nnz:     3602642
success: true

 SuiteSparse.CHOLMOD.Factor{Float64}
type:    LLt
method:  supernodal
maxnnz:  0
nnz:     3602642
success: true

 SuiteSparse.CHOLMOD.Factor{Float64}
type:    LLt
method:  supernodal
maxnnz:  0
nnz:     3602642
success: true

 SuiteSparse.CHOLMOD.Factor{Float64}
type:    LLt
method:  supernodal
maxnnz:  0
nnz:     3602642
success: true

 SuiteSparse.CHOLMOD.Factor{Float64}
ty

In [38]:
lop[1]

(M̃ = 
  [1    ,     1]  =  11.5809
  [2    ,     1]  =  0.801039
  [3    ,     1]  =  -0.820124
  [4    ,     1]  =  0.337643
  [5    ,     1]  =  -0.0274042
  [6    ,     1]  =  -0.0128202
  [7    ,     1]  =  -1.09617e-18
  [9    ,     1]  =  3.42553e-20
  [130  ,     1]  =  0.801039
  [259  ,     1]  =  -0.820124
  [388  ,     1]  =  0.337643
  [517  ,     1]  =  -0.0274042
  ⋮
  [15996, 16641]  =  -0.0128202
  [16125, 16641]  =  -0.0274042
  [16254, 16641]  =  0.337643
  [16383, 16641]  =  -0.820124
  [16512, 16641]  =  0.801039
  [16633, 16641]  =  3.42553e-20
  [16635, 16641]  =  -1.09617e-18
  [16636, 16641]  =  -0.0128202
  [16637, 16641]  =  -0.0274042
  [16638, 16641]  =  0.337643
  [16639, 16641]  =  -0.820124
  [16640, 16641]  =  0.801039
  [16641, 16641]  =  11.5809, F = (
  [1    ,   1]  =  -6.06879
  [2    ,   1]  =  -1.2638
  [3    ,   1]  =  0.947847
  [4    ,   1]  =  -0.421265
  [5    ,   1]  =  0.0789873
  [130  ,   2]  =  -26.7069
  [131  ,   2]  =  -5.56157
  [13

In [39]:
FToδstarts = bcstarts(FToB, FToE, FToLF, BC_JUMP_INTERFACE, Nr, Ns);

In [40]:
VNp = vstarts[nelems+1] - 1
λNp = FToλstarts[nfaces+1] - 1
δNp = FToδstarts[nfaces+1] - 1

0

In [41]:
vstarts

17-element Array{Int64,1}:
      1
  16642
  33283
  49924
  66565
  83206
  99847
 116488
 133129
 149770
 166411
 183052
 199693
 216334
 232975
 249616
 266257

In [42]:
@benchmark B = assembleλmatrix(FToλstarts,vstarts,EToF,FToB,locfactors,D,FbarT)

BenchmarkTools.Trial: 
  memory estimate:  15.80 GiB
  allocs estimate:  206099262
  --------------
  minimum time:     33.368 s (4.35% GC)
  median time:      33.368 s (4.35% GC)
  mean time:        33.368 s (4.35% GC)
  maximum time:     33.368 s (4.35% GC)
  --------------
  samples:          1
  evals/sample:     1

In [47]:
@benchmark B = threaded_assembleλmatrix(FToλstarts,vstarts,EToF,FToB,locfactors,D,FbarT)

BenchmarkTools.Trial: 
  memory estimate:  15.80 GiB
  allocs estimate:  206099262
  --------------
  minimum time:     31.832 s (4.41% GC)
  median time:      31.832 s (4.41% GC)
  mean time:        31.832 s (4.41% GC)
  maximum time:     31.832 s (4.41% GC)
  --------------
  samples:          1
  evals/sample:     1

 assembleλmatrix can not be parallelized easily with @threads despite it has for loop over elements

In [48]:
B = assembleλmatrix(FToλstarts,vstarts,EToF,FToB,locfactors,D,FbarT)

3096×3096 SparseMatrixCSC{Float64,Int64} with 2130048 stored entries:
  [1   ,    1]  =  6.95753
  [2   ,    1]  =  0.808076
  [3   ,    1]  =  -0.76754
  [4   ,    1]  =  0.245415
  [5   ,    1]  =  0.027543
  [6   ,    1]  =  -0.022876
  [7   ,    1]  =  -0.000763745
  [8   ,    1]  =  -1.99204e-5
  [9   ,    1]  =  -2.22388e-5
  [10  ,    1]  =  -7.66852e-6
  [11  ,    1]  =  -2.96721e-6
  [12  ,    1]  =  -1.29855e-6
  ⋮
  [3084, 3096]  =  -6.14113e-7
  [3085, 3096]  =  -1.29855e-6
  [3086, 3096]  =  -2.96721e-6
  [3087, 3096]  =  -7.66852e-6
  [3088, 3096]  =  -2.22388e-5
  [3089, 3096]  =  -1.99204e-5
  [3090, 3096]  =  -0.000763745
  [3091, 3096]  =  -0.022876
  [3092, 3096]  =  0.027543
  [3093, 3096]  =  0.245415
  [3094, 3096]  =  -0.76754
  [3095, 3096]  =  0.808076
  [3096, 3096]  =  6.95753

In [49]:
BF = cholesky(Symmetric(B))

SuiteSparse.CHOLMOD.Factor{Float64}
type:    LLt
method:  supernodal
maxnnz:  0
nnz:     2181519
success: true


In [50]:
@benchmark BF = cholesky(Symmetric(B))

BenchmarkTools.Trial: 
  memory estimate:  167.96 MiB
  allocs estimate:  39
  --------------
  minimum time:     121.696 ms (0.00% GC)
  median time:      137.611 ms (0.00% GC)
  mean time:        142.831 ms (0.21% GC)
  maximum time:     192.064 ms (0.51% GC)
  --------------
  samples:          36
  evals/sample:     1

In [79]:
@doc Symmetric

```
Symmetric(A, uplo=:U)
```

Construct a `Symmetric` view of the upper (if `uplo = :U`) or lower (if `uplo = :L`) triangle of the matrix `A`.

# Examples

```jldoctest
julia> A = [1 0 2 0 3; 0 4 0 5 0; 6 0 7 0 8; 0 9 0 1 0; 2 0 3 0 4]
5×5 Array{Int64,2}:
 1  0  2  0  3
 0  4  0  5  0
 6  0  7  0  8
 0  9  0  1  0
 2  0  3  0  4

julia> Supper = Symmetric(A)
5×5 Symmetric{Int64,Array{Int64,2}}:
 1  0  2  0  3
 0  4  0  5  0
 2  0  7  0  8
 0  5  0  1  0
 3  0  8  0  4

julia> Slower = Symmetric(A, :L)
5×5 Symmetric{Int64,Array{Int64,2}}:
 1  0  6  0  2
 0  4  0  9  0
 6  0  7  0  3
 0  9  0  1  0
 2  0  3  0  4
```

Note that `Supper` will not be equal to `Slower` unless `A` is itself symmetric (e.g. if `A == transpose(A)`).


In [80]:
(bλ,λ,gδ) = (zeros(λNp), zeros(λNp), zeros(λNp)) 

([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [81]:
(Δ,u,g) = (zeros(VNp), zeros(VNp), zeros(VNp))

([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [82]:
δ = zeros(δNp)

0-element Array{Float64,1}

In [83]:
for f = 1:nfaces
    if FToB[f] == BC_JUMP_INTERFACE
        (e1, e2) = FToE[:,f]
        (lf1, lf2) = FToLF[:,f]
        (xf, yf) = lop[e1].facecoord
        @views δ[FToδstarts[f]:(FToδstarts[f+1]-1)] = 
            vex(xf[lf1],yf[lf1],e2) - vex(xf[lf1],yf[lf1],e1)
    end
end

In [84]:
bc_Dirichlet = (lf, x, y, e, δ) -> vex(x,y,e)
bc_Neumann   = (lf, x, y, nx, ny, e, δ) -> (nx .* vex_x(x,y,e) + ny .* vex(x,y,e))

#277 (generic function with 1 method)

In [85]:
in_jump      = (lf, x, y, e, δ) -> begin
    f = EToF[lf, e]
    if EToS[lf, e] == 1
        if EToO[lf, e]
            return -δ[FToδstarts[f]:(FToδstarts[f+1]-1)]
        else
            error("shouldn't get here")
        end
    else
        if EToO[lf,e]
            return δ[FToδstarts[f]:(FToδstarts[f+1]-1)]
        else
            return δ[(FToδstarts[f+1]-1):-1:FToδstarts[f]]
        end
    end
end
    

#279 (generic function with 1 method)

In [86]:
# @benchmark 
for e = 1:nelems
      gδe = ntuple(4) do lf
        f = EToF[lf, e]
        if EToO[lf, e]
          return @view gδ[FToλstarts[f]:(FToλstarts[f+1]-1)]
        else
          return  @view gδ[(FToλstarts[f+1]-1):-1:FToλstarts[f]]
        end
      end
      locbcarray!((@view g[vstarts[e]:vstarts[e+1]-1]), gδe, lop[e],
                  FToB[EToF[:,e]], bc_Dirichlet, bc_Neumann, in_jump, (e, δ))

      source = (x, y, e) -> (-vex_xx(x, y, e)  - vex_yy(x, y, e))
      locsourcearray!((@view g[vstarts[e]:vstarts[e+1]-1]), source, lop[e], e)
end


In [87]:

@threads for e = 1:nelems
      gδe = ntuple(4) do lf
        f = EToF[lf, e]
        if EToO[lf, e]
          return @view gδ[FToλstarts[f]:(FToλstarts[f+1]-1)]
        else
          return  @view gδ[(FToλstarts[f+1]-1):-1:FToλstarts[f]]
        end
      end
      locbcarray!((@view g[vstarts[e]:vstarts[e+1]-1]), gδe, lop[e],
                  FToB[EToF[:,e]], bc_Dirichlet, bc_Neumann, in_jump, (e, δ))

      source = (x, y, e) -> (-vex_xx(x, y, e)  - vex_yy(x, y, e))
      locsourcearray!((@view g[vstarts[e]:vstarts[e+1]-1]), source, lop[e], e)
end


In [88]:
LocalToGLobalRHS!(bλ, g, gδ,  u, locfactors, FbarT, vstarts)

3096-element Array{Float64,1}:
 1940.4657943660495     
  859.955213207742      
 -144.56456569130378    
  243.93951751909827    
   67.02128303745845    
   75.19413121482606    
   64.35031637273443    
   57.14497672463439    
   52.01883881055562    
   48.23180065022449    
   45.37218133901549    
   43.18143017681698    
   41.48834834481405    
    ⋮                   
    6.3518293511688935  
    6.268808152202747   
    6.145746315188002   
    5.975137128290921   
    5.747579814469819   
    5.4509835338403585  
    5.139827300921717   
    4.17638627562361    
    4.897093666042742   
    1.969053385358868   
    2.745755111414145   
    0.052214434065504495

In [89]:
@benchmark LocalToGLobalRHS!(bλ, g, gδ,  u, locfactors, FbarT, vstarts)

BenchmarkTools.Trial: 
  memory estimate:  8.22 MiB
  allocs estimate:  208
  --------------
  minimum time:     108.817 ms (0.00% GC)
  median time:      112.322 ms (0.00% GC)
  mean time:        113.837 ms (0.06% GC)
  maximum time:     132.732 ms (0.00% GC)
  --------------
  samples:          44
  evals/sample:     1

In [90]:
@benchmark threaded_LocalToGLobalRHS!(bλ, g, gδ, u, locfactors, FbarT, vstarts)

BenchmarkTools.Trial: 
  memory estimate:  8.22 MiB
  allocs estimate:  238
  --------------
  minimum time:     45.924 ms (0.00% GC)
  median time:      49.702 ms (0.00% GC)
  mean time:        50.289 ms (0.00% GC)
  maximum time:     66.306 ms (0.00% GC)
  --------------
  samples:          100
  evals/sample:     1

Conclusion: threaded_LocalToGLobalRHS! would help but not too much

In [91]:
@show bλ
@show g
@show gδ
@show u
@show locfactors # M
@show FbarT 
@show vstarts

bλ = [1940.4657943660495, 859.955213207742, -144.56456569130378, 243.93951751909827, 67.02128303745845, 75.19413121482606, 64.35031637273443, 57.14497672463439, 52.01883881055562, 48.23180065022449, 45.37218133901549, 43.18143017681698, 41.48834834481405, 40.17504449472775, 39.15766447855237, 38.37494650730402, 37.78111436115151, 37.34129749192685, 37.02848581337749, 36.82144994475998, 36.70328738434069, 36.66038523130947, 36.68166654331871, 36.75803379226847, 36.881951795912364, 37.047130978529964, 37.24828388042926, 37.48093587352418, 37.741276490463335, 38.02604153129491, 38.332418739730365, 38.657971705148256, 39.0005779855419, 39.35837841996612, 39.72973531431494, 40.11319771533076, 40.50747238580161, 40.91139939497404, 41.323931467837745, 41.74411641347652, 42.17108208942516, 42.60402346565022, 43.042191435544034, 43.484883087544745, 43.931433203648744, 44.38120679320038, 44.83359250421951, 45.2879967819186, 45.74383866632178, 46.200545139086636, 46.65754694454362, 47.11427482227

Excessive output truncated after 5705710 bytes.

g = 

17-element Array{Int64,1}:
      1
  16642
  33283
  49924
  66565
  83206
  99847
 116488
 133129
 149770
 166411
 183052
 199693
 216334
 232975
 249616
 266257

In [92]:
for e=1:length(locfactors)
    @views u[vstarts[e]:(vstarts[e+1]-1)] = (locfactors[e] \ g[vstarts[e]:(vstarts[e+1]-1)])
end

In [93]:
mul!(bλ,FbarT,u)

3096-element Array{Float64,1}:
 -1940.4657943660495     
  -859.955213207742      
   144.56456569130378    
  -243.93951751909827    
   -67.02128303745845    
   -75.19413121482606    
   -64.35031637273443    
   -57.14497672463439    
   -52.01883881055562    
   -48.23180065022449    
   -45.37218133901549    
   -43.18143017681698    
   -41.48834834481405    
     ⋮                   
    -6.3518293511688935  
    -6.268808152202747   
    -6.145746315188002   
    -5.975137128290921   
    -5.747579814469819   
    -5.4509835338403585  
    -5.139827300921717   
    -4.17638627562361    
    -4.897093666042742   
    -1.969053385358868   
    -2.745755111414145   
    -0.052214434065504495

In [94]:
@show typeof(bλ)
@show typeof(FbarT)
@show typeof(u)

typeof(bλ) = Array{Float64,1}
typeof(FbarT) = SparseMatrixCSC{Float64,Int64}
typeof(u) = Array{Float64,1}


Array{Float64,1}

In [95]:
@doc mul!

```
mul!(Y, A, B) -> Y
```

Calculates the matrix-matrix or matrix-vector product $AB$ and stores the result in `Y`, overwriting the existing value of `Y`. Note that `Y` must not be aliased with either `A` or `B`.

# Examples

```jldoctest
julia> A=[1.0 2.0; 3.0 4.0]; B=[1.0 1.0; 1.0 1.0]; Y = similar(B); mul!(Y, A, B);

julia> Y
2×2 Array{Float64,2}:
 3.0  3.0
 7.0  7.0
```

# Implementation

For custom matrix and vector types, it is recommended to implement 5-argument `mul!` rather than implementing 3-argument `mul!` directly if possible.

```
mul!(C, A, B, α, β) -> C
```

Combined inplace matrix-matrix or matrix-vector multiply-add $A B α + C β$. The result is stored in `C` by overwriting it.  Note that `C` must not be aliased with either `A` or `B`.

!!! compat "Julia 1.3"
    Five-argument `mul!` requires at least Julia 1.3.


# Examples

```jldoctest
julia> A=[1.0 2.0; 3.0 4.0]; B=[1.0 1.0; 1.0 1.0]; C=[1.0 2.0; 3.0 4.0];

julia> mul!(C, A, B, 100.0, 10.0) === C
true

julia> C
2×2 Array{Float64,2}:
 310.0  320.0
 730.0  740.0
```


In [46]:
@benchmark λ[:] = BF \ bλ

UndefVarError: UndefVarError: BF not defined

In [97]:
u[:] = -FbarT' * λ

266256-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮  
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [98]:
u[:] .= g .+ u

266256-element view(::Array{Float64,1}, :) with eltype Float64:
 1.7402080323534172e6
 3.8267805815270967e6
 1.4306964391391946e6
 3.0088406031459873e6
 2.0517795812066875e6
 2.184519911731605e6 
 2.051338580671833e6 
 1.9530752097934955e6
 1.8595188581016746e6
 1.7704440496186782e6
 1.6856361091336065e6
 1.6048906448232317e6
 1.5280130556564063e6
 ⋮                   
 0.0890929077617104  
 0.09357527173480022 
 0.098283157283931   
 0.10322791067344353 
 0.1084214490322637  
 0.11387628907484165 
 0.12126953448790209 
 0.11452932038615814 
 0.16367703968945654 
 0.08696644403530006 
 0.20237734058975668 
 0.048301371355733376

In [99]:
time_direct_solve_start = time()
# @benchmark 
for e = 1:nelems
      F = locfactors[e]
      (x, y) = lop[e].coord
      JH = lop[e].JH
      start = time()
      @views u[vstarts[e]:(vstarts[e+1]-1)] = F \ u[vstarts[e]:(vstarts[e+1]-1)]
      #=
      ldiv!((@view u[vstarts[e]:(vstarts[e+1]-1)]), F,
    
            (@view u[vstarts[e]:(vstarts[e+1]-1)]))
      =#

      @views Δ[vstarts[e]:(vstarts[e+1]-1)] = (u[vstarts[e]:(vstarts[e+1]-1)] -
                                               vex(x[:], y[:], e))
      ϵ[lvl] += Δ[vstarts[e]:(vstarts[e+1]-1)]' * JH * Δ[vstarts[e]:(vstarts[e+1]-1)]
 end
time_direct_solve_elapsed = time() - time_direct_solve_start

0.14700007438659668

In [100]:
time_direct_solve_threaded_start = time()
# @benchmark 
@threads for e = 1:nelems
      F = locfactors[e]
      (x, y) = lop[e].coord
      JH = lop[e].JH
      start = time()
      @views u[vstarts[e]:(vstarts[e+1]-1)] = F \ u[vstarts[e]:(vstarts[e+1]-1)]
      #=
      ldiv!((@view u[vstarts[e]:(vstarts[e+1]-1)]), F,
            (@view u[vstarts[e]:(vstarts[e+1]-1)]))
      =#

      @views Δ[vstarts[e]:(vstarts[e+1]-1)] = (u[vstarts[e]:(vstarts[e+1]-1)] -
                                               vex(x[:], y[:], e))
      ϵ[lvl] += Δ[vstarts[e]:(vstarts[e+1]-1)]' * JH * Δ[vstarts[e]:(vstarts[e+1]-1)]
 end
time_direct_solve_threaded_elapsed = time() - time_direct_solve_threaded_start

0.08999991416931152

Conclusion: using Multithreading would significantly boost the performance of parallel solve

In [ ]:
nthreads()